In [1]:
import os
import cv2
import pandas as pd
import math
import numpy as np
import warnings
warnings.filterwarnings("ignore")

GAME_STATE_FILE_NAME = "game"
GAME_STATE_FILE_EXT = ".csv"
GAMES_DIR = "games/"
PROCESSED_GAMES_DIR = "processed_games/"
MODEL_NAME = "2048_model.h5"
MOVES = ["UP", "DOWN", "LEFT", "RIGHT"]
MOVE_COL_NAME = "MOVE"
N_SIZE = 4
N_FILES = len(os.listdir(PROCESSED_GAMES_DIR))
TRAIN_MODEL = True

def load_data(file, direc=GAMES_DIR, header=True):
    csv_path = os.path.join(direc, file)
    if header:
        return pd.read_csv(csv_path)
    else:
        return pd.read_csv(csv_path, header=None)

In [2]:
def get_features_labels(n_file, direc, group_n_games=N_FILES, validation=False):
    x = []
    y = []
    
    if not validation:
        group_n_games = 1
        
    for indx in range(group_n_games):
        
        filename = GAME_STATE_FILE_NAME + str(n_file % N_FILES) + GAME_STATE_FILE_EXT
        n_file = n_file - 1
        
        data = load_data(file=filename, direc=direc)
    
        labels = data[MOVE_COL_NAME].values
        data.drop(MOVE_COL_NAME, axis=1, inplace=True)
        #binarizer = LabelBinarizer()
        #binarizer.fit([0, 1, 2, 3])
        #labels = binarizer.transform(labels)

        features = data.values
        #features = np.reshape(features, (-1, N_SIZE, N_SIZE, 1))
        
        if len(x) == 0:
            x = features
            y = labels
        else:
            x = np.concatenate((x, features), axis=0)
            y = np.concatenate((y, labels), axis=0)
                                              
    return x, y

In [3]:
# Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import cross_val_score
from sklearn.base import clone
from sklearn.model_selection import GridSearchCV

parameters = {
    'max_depth' : [9, 10, 11, 12, 13, 14, 15, 16],
}

clf = GridSearchCV(RandomForestClassifier(random_state=42), parameters)

features, labels = get_features_labels(1, direc=PROCESSED_GAMES_DIR)
val_features, val_labels = get_features_labels(0, group_n_games=N_FILES - 1, direc=PROCESSED_GAMES_DIR, validation=True)

clf.fit(val_features, val_labels)
print(clf.best_params_)

forest_clf = RandomForestClassifier(random_state=42, max_depth=16)
print("Cross Val Scores on training set\n", np.mean(cross_val_score(clone(forest_clf), features, labels, cv=N_FILES, scoring="accuracy")))

forest_clf.fit(val_features, val_labels)
print("\n\nAccuracy on testing data set\n", sum(val_labels == forest_clf.predict(val_features)) / len(val_labels))

{'max_depth': 16}
Cross Val Scores on training set
 0.3914329268292682


Accuracy on testing data set
 0.7509140888047138
